In [14]:
import pyodbc
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import xgboost

conn = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=JUANBECETI;'
                      'DATABASE=Principal;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

# Ejecutar una consulta SQL para seleccionar los datos de la tabla
cursor.execute('SELECT * FROM bank_marketing')
rows = cursor.fetchall()

columns = [column[0] for column in cursor.description]

marketing = pd.DataFrame.from_records(rows, columns=columns)

cursor.close()
conn.close()

#LIMPIEZA DE DATOS

marketing.drop(columns = "nr_employed", inplace = True)

marketing["housing"] = marketing["housing"].fillna("True")
marketing["loan"] = marketing["loan"].fillna("False")
marketing["euribor3m"] = marketing["euribor3m"].fillna(3286.9)

marketing["y"] = np.where(marketing["y"] == True, 1, 0)
marketing["housing"] = np.where(marketing["housing"] == False, 0, 1)

def edad(dataframe, variable):
     if variable <= 20:
          return "Menor de 20"
     elif  20 < variable <= 30:
          return "Entre 20 y 30"
     elif 30 < variable <= 40:
          return "Entre 30 y 40"
     elif 40 < variable <= 50:
          return "Entre 40 y 50"
     elif 50 < variable <= 60:
          return "Entre 50 y 60"
     elif 60 < variable:
          return "Mayor de 60"
     return dataframe

marketing["Range_age"] = marketing["age"].apply(lambda x: edad(marketing,x))


# MODELO XGBOOST

df = marketing.copy()
y = df["y"]
df.drop(columns=["age", "loan", "y"], inplace=True)
column_names = df.select_dtypes(include=['number']).columns.tolist()


        # TRANSFORMACIONES A VARIABLES NUMERICAS

min_max =  MinMaxScaler(feature_range=(0, 1))
df_numeric = df.select_dtypes("number")
df_numeric = min_max.fit_transform(df_numeric)
df_numeric = pd.DataFrame(data = df_numeric, columns = column_names).reset_index(drop = True)

        # TRANSFORMACIONES A VARIABLES CATEGORICAS

ohe = OneHotEncoder(sparse = False)
df_cat = df.select_dtypes("O")
ohe.fit(df_cat)
df_cat_ohe = ohe.transform(df_cat)
df_cat_ohe = pd.DataFrame(df_cat_ohe, columns = ohe.get_feature_names_out(input_features = df_cat.columns)).reset_index(drop = True)

df = pd.concat([df_cat_ohe, df_numeric], axis = 1)
x = df.copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

xgb_model = xgboost.XGBClassifier()

        # DEFINIR LOS PARÁMETROS DEL MODELO
params = {
    'objective': 'binary:logistic',  # Para problemas de clasificación binaria
    'max_depth': 5,                  # Profundidad máxima del árbol
    'min_child_weight': 1,           # Peso mínimo requerido en un nodo hijo
    'gamma': 0.1,                    # Parámetro de penalización para la división de nodos
    'eval_metric': 'error'           # Métrica de evaluación, aquí se usa tasa de error
}

xgb_model.set_params(**params)
xgb_model.fit(x_train, y_train)

marketing["Prob_deposito"] = xgb_model.predict_proba(df)[:,1]

c:\Users\pablo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [18]:
import pyodbc
import pandas as pd

# Ejemplo de conexión (ajusta los parámetros según tu configuración)

conn = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=JUANBECETI;'
                      'DATABASE=Principal;'
                      'Trusted_Connection=yes;')

# Cursor para ejecutar comandos SQL
cursor = conn.cursor()

# Ejemplo de DataFrame (reemplaza con tu DataFrame real)


df = pd.DataFrame(marketing)

# Nombre de la tabla en SQL Server
table_name = 'Marketing_bank_model'

# Crear una sentencia SQL para borrar y crear la tabla
sql_query = f'''
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name} (
    age INT,
    job VARCHAR(100),
    marital VARCHAR(100),
    education VARCHAR(100),
    [default] VARCHAR(100),
    housing INT,
    loan VARCHAR(100),
    contact VARCHAR(100),
    month VARCHAR(100),
    day_of_week VARCHAR(100),
    duration INT,
    campaign INT,
    pdays INT,
    previous INT,
    poutcome VARCHAR(100),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    cons_conf_idx FLOAT,
    euribor3m FLOAT,
    y INT,
    Range_age VARCHAR(100),
    Prob_deposito FLOAT
);
'''

# Ejecutar la sentencia SQL
cursor.execute(sql_query)

# Insertar datos en la tabla desde el DataFrame
for index, row in df.iterrows():
    cursor.execute(f'''
    INSERT INTO {table_name} (age, job, marital, education, [default], housing, loan,
                              contact, month, day_of_week, duration, campaign, pdays,
                              previous, poutcome, emp_var_rate, cons_price_idx,
                              cons_conf_idx, euribor3m, y, Range_age, Prob_deposito)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(row))

# Confirmar la transacción y cerrar la conexión
conn.commit()
conn.close()

print(f'Tabla {table_name} creada y datos insertados correctamente.')

Tabla Marketing_bank_model creada y datos insertados correctamente.
